In [28]:
keyboard_low = [[ '`', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '=', '<back>'],
                [ '\t', 'q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '[', ']', "\\"],
                ['<caps>','a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';', "'", '\n'],
                ['<shift>','z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/', '<shift>'],
                ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

keyboard_high = [[ '~', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+', '<back>'],
                 [ '\t', 'Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', '{', '}', "|"],
                 ['<caps>','A','S','D','F','G','H', 'J', 'K', 'L', ':', '"', '\n'],
                 ['<shift>','Z', 'X', 'C', 'V', 'B', 'N', 'M', '<', '>', '?', '<shift>',],
                 ['<win>', '<alt>',' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>' ]]
                

In [29]:
def generate_dict(keyboard_low, keyboard_high):
    low_dict = {}
    high_dict = {}

    for i in range(len(keyboard_low)):
        for j in range(len(keyboard_low[i])):
            key = keyboard_low[i][j]
            if key in low_dict:
                low_dict[key].append([i, j])
            else:
                low_dict[key] = [[i, j]]

    for i in range(len(keyboard_high)):
        for j in range(len(keyboard_high[i])):
            key = keyboard_high[i][j]
            if key in high_dict:
                high_dict[key].append([i, j])
            else:
                high_dict[key] = [[i, j]]
    return low_dict, high_dict

low_dict, high_dict =generate_dict(keyboard_low, keyboard_high)
low_dict['a']

[[2, 1]]

In [30]:
low_dict['<shift>']

[[3, 0], [3, 11]]

In [31]:
keyboard_high

[['~', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+', '<back>'],
 ['\t', 'Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', '{', '}', '|'],
 ['<caps>', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ':', '"', '\n'],
 ['<shift>', 'Z', 'X', 'C', 'V', 'B', 'N', 'M', '<', '>', '?', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

In [131]:
from math import sqrt
import numpy as np
from copy import deepcopy

class finger:
    def __init__(self, default, name) -> None:
        self.name = name
        self.default = default
        self.counter = 0
        self.current = default
        self.typed_keys = 0

    def move_to(self, cell):
        self.current = cell
        self.counter = 4
        self.typed_keys += 1

    def tick(self):
        if self.counter > 0:
            self.counter -= 1
        else:
            return 0
        if self.counter == 0:
            distance = self.distance_to(self.default)
            self.current = self.default
            return distance
        return 0

    def distance_to(self, target):
        x1, y1 = self.current
        x2, y2 = target
        offset = 0
        if abs(y1-y2) > 3:
            offset = 1
        return sqrt(abs(x1-x2)**2 + abs((y1-y2)*1.2)**2+offset)
    
    def reset(self):
        self.current = self.default
        self.typed_keys = 0
        self.counter = 0
    
    def print_stat(self):
        print(f"Name: {self.name}\t\t Typed keys: {self.typed_keys}\t Current counter: {self.counter}\t Current position: {self.current}\t Default position: {self.default}")



class keyboard:
    def __init__(self, keyboard_low, keyboard_high, low_dict, high_dict, show = False):
        self.keyboard_low = deepcopy(keyboard_low)
        self.keyboard_high = deepcopy(keyboard_high)
        self.low_dict = deepcopy(low_dict)
        self.high_dict = deepcopy(high_dict)
        self.show = show
        self.fingers = []
        f0 = finger([2, 1], "левый мизинец")
        f1 = finger([2, 2], "левый безымянный")
        f2 = finger([2, 3], "левый средний")
        f3 = finger([2, 4], "левый указательный")
        f4 = finger([4, 3], "левый большой")
        f5 = finger([4, 6], "правый большой")
        f6 = finger([2, 7], "правый указательный")
        f7 = finger([2, 8], "правый средний")
        f8 = finger([2, 9], "правый безымянный")
        f9 = finger([2, 10], "правый мизинец")
        self.fingers.extend([f0, f1, f2, f3, f4, f5, f6, f7, f8, f9])
        self.total_distance = 0

    def move_one_finger(self, cells, banned = -1):
        best_finger_id = 0
        best_distance = 99999
        final_cell = [0, 0]
        for cell in cells:
            distances = []
            for i in range(10):
                if i != banned:
                    distances.append(self.fingers[i].distance_to(cell))
                else:
                    distances.append(99999)
            candidate_finger_id = np.argmin(distances)
            candidate_distance = min(distances)
            if candidate_distance < best_distance:
                best_distance = candidate_distance
                best_finger_id = candidate_finger_id
                final_cell = cell
        return best_finger_id, final_cell, best_distance

    def move_two_fingers(self, cells):

        distance = 0

        f1_shift, c1_shift, d1_shift = self.move_one_finger(self.low_dict['<shift>'])
        f1_key, c1_key, d1_key = self.move_one_finger(cells, banned=f1_shift)

        f2_key, c2_key, d2_key = self.move_one_finger(cells)
        f2_shift, c2_shift, d2_shift = self.move_one_finger(self.low_dict['<shift>'], f2_key)

        if d1_shift + d1_key < d2_key + d2_shift:
            distance = d1_shift + d1_key 
            return f1_shift, f1_key, c1_shift, c1_key, distance
        else:
            distance = d2_key + d2_shift
            return f2_key, f2_shift, c2_key, c2_shift, distance

    def end_move(self):
        for finger in self.fingers:
            self.total_distance += finger.tick()
    
    def find_key(self, key):
        if key in self.low_dict:
            finger_id, cell, distance = self.move_one_finger(self.low_dict[key], self.fingers)
            self.fingers[finger_id].move_to(cell)
            self.total_distance += distance
            if self.show:
                print(key + ': ' + self.fingers[finger_id].name)
        elif key in self.high_dict:
            f1_id, f2_id, f1_cell, f2_cell, distance = self.move_two_fingers(self.high_dict[key])
            self.fingers[f1_id].move_to(f1_cell)
            self.fingers[f2_id].move_to(f2_cell)
            self.total_distance += distance
            if self.show:
                print(key + ': ' + self.fingers[f1_id].name + "+" + self.fingers[f2_id].name)
        else:
            print("THERE IS NO SUCH KEY: ", key)
        self.end_move()
    
    def type_string(self, str):
        for letter in str:
            self.find_key(letter)
        return self.total_distance
    
    def swap_cells_low(self, cell1, cell2):
        x1, y1 = cell1
        key1 = self.keyboard_low[x1][y1]
        x2, y2 = cell2
        key2 = self.keyboard_low[x2][y2]
        self.low_dict[key1].remove(cell1)
        self.low_dict[key2].remove(cell2)

        self.low_dict[key1].append(cell2)
        self.low_dict[key2].append(cell1)

        self.keyboard_low[x2][y2] = key1
        self.keyboard_low[x1][y1] = key2

    def swap_cells_high(self, cell1, cell2):
        x1, y1 = cell1
        key1 = self.keyboard_high[x1][y1]
        x2, y2 = cell2
        key2 = self.keyboard_high[x2][y2]
        self.high_dict[key1].remove(cell1)
        self.high_dict[key2].remove(cell2)

        self.high_dict[key1].append(cell2)
        self.high_dict[key2].append(cell1)

        self.keyboard_high[x2][y2] = key1
        self.keyboard_high[x1][y1] = key2

    def swap_cells_diff(self, cell_low, cell_high):
        x1, y1 = cell_low
        key_low = self.keyboard_low[x1][y1]
        x2, y2 = cell_high
        key_high = self.keyboard_high[x2][y2]
        self.low_dict[key_low].remove(cell_low)
        self.high_dict[key_high].remove(cell_high)

        if key_high in self.low_dict:
            self.low_dict[key_high].append(cell_low)
        else:
            self.low_dict[key_high] = [cell_low]
        
        if key_low in self.high_dict:
            self.high_dict[key_low].append(cell_high)
        else:
            self.high_dict[key_low] = [cell_high]

        self.keyboard_high[x2][y2] = key_low
        self.keyboard_low[x1][y1] = key_high

    def reset_keyboard(self):
        self.total_distance = 0
        for f in self.fingers:
            f.reset()
    
    def show_fingers_stat(self):
        for f in self.fingers:
            f.print_stat()


In [132]:
all_keys = []
keyboard_shape = []

for i in range(len(keyboard_low)):
    keyboard_shape.append(len(keyboard_low[i]))
    for j in range(len(keyboard_low[i])):
        key = keyboard_high[i][j]
        if key != '<shift>':
            all_keys.append(key)

for i in range(len(keyboard_high)):
    for j in range(len(keyboard_high[i])):
        key = keyboard_high[i][j]
        if key != '<shift>':
            all_keys.append(key)

In [133]:
import random 

def generate_cell(keyboard_shape):
    x = random.randint(0, len(keyboard_shape)-1)
    y = random.randint(0, keyboard_shape[x]-1)
    return x, y

def generate_layout(all_keys, keyboard_shape):
    low = [[0 for _ in range(row)] for row in keyboard_shape] 
    high = [[0 for _ in range(row)] for row in keyboard_shape] 
    x1, y1 = generate_cell(keyboard_shape)
    x2, y2 = x1, y1
    while x1 == x2 and y1 == y2:
        x2, y2 = generate_cell(keyboard_shape)
    low[x1][y1] = '<shift>'
    high[x1][y1] = '<shift>'
    low[x2][y2] = '<shift>'
    high[x2][y2] = '<shift>'
    
    temp = deepcopy(all_keys)
    random.shuffle(temp)
    
    for i in range(len(low)):
        for j in range(len(low[i])):
            key = low[i][j]
            if key == '<shift>':
                continue
            key = temp.pop()
            low[i][j] = key
    
    for i in range(len(high)):
        for j in range(len(high[i])):
            key = high[i][j]
            if key == '<shift>':
                continue
            key = temp.pop()
            high[i][j] = key

    return low, high


generate_layout(all_keys, keyboard_shape)

([['*', '*', '_', 'V', '\n', 'B', ' ', 'X', 'N', 'B', '_', 'A', ' ', '<fn>'],
  ['S',
   '}',
   '!',
   'I',
   'L',
   ' ',
   '<caps>',
   '#',
   'K',
   '<win>',
   '|',
   'G',
   ' ',
   '?'],
  [' ', ' ', 'V', 'S', 'T', '<back>', '}', '<shift>', '!', 'O', 'L', 'W', '{'],
  ['U',
   'P',
   'M',
   '<alt>',
   'O',
   '<caps>',
   '\t',
   '<alt>',
   '"',
   '<win>',
   'N',
   '<'],
  ['<shift>', ' ', 'W', '@', '@', 'C', '$', 'F', '(', 'Q']],
 [['?', ')', '%', ' ', 'E', 'U', 'Q', '|', 'G', 'Y', '%', ' ', 'D', 'R'],
  ['P', '+', 'T', '^', '#', ')', '$', ':', ' ', '\n', 'H', 'Z', '+', '&'],
  [':', 'Y', '<', '~', '&', '^', '>', '<shift>', 'H', 'Z', 'F', '<alt>', 'D'],
  ['"', 'A', '<back>', 'R', '<fn>', '~', '\t', '<alt>', 'E', 'J', 'M', 'X'],
  ['<shift>', ' ', '>', 'K', '{', 'C', '(', ' ', 'J', 'I']])

In [134]:
len(all_keys)

122

In [135]:
keyboard_shape

[14, 14, 13, 12, 10]

In [136]:
k = keyboard(keyboard_low, keyboard_high, low_dict, high_dict, show=True)
k.low_dict['s'], k.low_dict['q'], k.high_dict['S'], k.high_dict['Q']

([[2, 2]], [[1, 1]], [[2, 2]], [[1, 1]])

In [137]:
k.swap_cells_low([1, 1], [2, 2])
k.swap_cells_high([1, 1], [2, 2])

In [138]:
k.low_dict['s'], k.low_dict['q'], k.high_dict['S'], k.high_dict['Q']

([[1, 1]], [[2, 2]], [[1, 1]], [[2, 2]])

In [139]:
k.keyboard_low

[['`', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '=', '<back>'],
 ['\t', 's', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '[', ']', '\\'],
 ['<caps>', 'a', 'q', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';', "'", '\n'],
 ['<shift>', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

In [140]:
k.keyboard_high

[['~', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+', '<back>'],
 ['\t', 'S', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', '{', '}', '|'],
 ['<caps>', 'A', 'Q', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ':', '"', '\n'],
 ['<shift>', 'Z', 'X', 'C', 'V', 'B', 'N', 'M', '<', '>', '?', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

In [141]:
k.low_dict['w'], k.high_dict['E']

([[1, 2]], [[1, 3]])

In [142]:
k.swap_cells_diff([1, 2], [1, 3])
k.keyboard_low

[['`', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '=', '<back>'],
 ['\t', 's', 'E', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '[', ']', '\\'],
 ['<caps>', 'a', 'q', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';', "'", '\n'],
 ['<shift>', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

In [143]:
k.keyboard_high

[['~', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+', '<back>'],
 ['\t', 'S', 'W', 'w', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', '{', '}', '|'],
 ['<caps>', 'A', 'Q', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ':', '"', '\n'],
 ['<shift>', 'Z', 'X', 'C', 'V', 'B', 'N', 'M', '<', '>', '?', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

In [144]:
k.low_dict['w'], k.high_dict['E'], k.high_dict['w'], k.low_dict['E'], 

([], [], [[1, 3]], [[1, 2]])

In [145]:
k.show_fingers_stat()

Name: левый мизинец		 Typed keys: 0	 Current counter: 0	 Current position: [2, 1]	 Default position: [2, 1]
Name: левый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 2]	 Default position: [2, 2]
Name: левый средний		 Typed keys: 0	 Current counter: 0	 Current position: [2, 3]	 Default position: [2, 3]
Name: левый указательный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 4]	 Default position: [2, 4]
Name: левый большой		 Typed keys: 0	 Current counter: 0	 Current position: [4, 3]	 Default position: [4, 3]
Name: правый большой		 Typed keys: 0	 Current counter: 0	 Current position: [4, 6]	 Default position: [4, 6]
Name: правый указательный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 7]	 Default position: [2, 7]
Name: правый средний		 Typed keys: 0	 Current counter: 0	 Current position: [2, 8]	 Default position: [2, 8]
Name: правый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 9]	 Default position: [2, 9]
Name: пр

In [146]:
k.type_string('Hi my name is Seva!')

H: правый указательный+левый мизинец
i: правый средний
 : левый большой
m: правый большой
y: правый указательный
 : левый большой
n: правый большой
a: левый мизинец
m: правый указательный
e: левый средний
 : левый большой
i: правый средний
s: левый мизинец
 : левый большой
S: левый мизинец+правый мизинец
e: левый средний
v: левый указательный
a: левый мизинец
!: левый мизинец+правый мизинец


32.096399481450646

In [147]:
k.show_fingers_stat()

Name: левый мизинец		 Typed keys: 6	 Current counter: 3	 Current position: [0, 1]	 Default position: [2, 1]
Name: левый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 2]	 Default position: [2, 2]
Name: левый средний		 Typed keys: 2	 Current counter: 0	 Current position: [2, 3]	 Default position: [2, 3]
Name: левый указательный		 Typed keys: 1	 Current counter: 1	 Current position: [3, 4]	 Default position: [2, 4]
Name: левый большой		 Typed keys: 4	 Current counter: 0	 Current position: [4, 3]	 Default position: [4, 3]
Name: правый большой		 Typed keys: 2	 Current counter: 0	 Current position: [4, 6]	 Default position: [4, 6]
Name: правый указательный		 Typed keys: 3	 Current counter: 0	 Current position: [2, 7]	 Default position: [2, 7]
Name: правый средний		 Typed keys: 2	 Current counter: 0	 Current position: [2, 8]	 Default position: [2, 8]
Name: правый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 9]	 Default position: [2, 9]
Name: пр

In [148]:
k.reset_keyboard()
k.show_fingers_stat()

Name: левый мизинец		 Typed keys: 0	 Current counter: 0	 Current position: [2, 1]	 Default position: [2, 1]
Name: левый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 2]	 Default position: [2, 2]
Name: левый средний		 Typed keys: 0	 Current counter: 0	 Current position: [2, 3]	 Default position: [2, 3]
Name: левый указательный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 4]	 Default position: [2, 4]
Name: левый большой		 Typed keys: 0	 Current counter: 0	 Current position: [4, 3]	 Default position: [4, 3]
Name: правый большой		 Typed keys: 0	 Current counter: 0	 Current position: [4, 6]	 Default position: [4, 6]
Name: правый указательный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 7]	 Default position: [2, 7]
Name: правый средний		 Typed keys: 0	 Current counter: 0	 Current position: [2, 8]	 Default position: [2, 8]
Name: правый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 9]	 Default position: [2, 9]
Name: пр

As we can see, we successfully can swap different keys in given keyboard and calculate distance function fro given string.

# Check

let's check on some program from this [dataset](https://www.kaggle.com/datasets/joshuwamiller/software-code-dataset-cc).

In [14]:
import pandas as pd

df = pd.read_csv("./validated/200717.cpp.csv", sep='`')
temp = df['code']
len(temp)


14

In [15]:
temp_sent = '\n'.join(temp)

In [16]:
temp_sent

'void update(bool flag[DIM][DIM], string type, string pos) {\n\tint x=pos[0]-\'a\', y=pos[1]-\'1\';\n\tif(type=="rook") {\n\t\tfor (int i=0; i<DIM; i++) {\n\t\t\tfor (int j=0; j<DIM; j++) {\n\t\t\t\tif((i==x || j==y) && !(i==x && j==y)) flag[i][j]=true;\n\t\tfor (int i=-1; i<=1; i++) {\n\t\t\tfor (int j=-1; j<=1; j++) {\n\t\t\t\tif(!(i==0 && j==0)) {\n\t\t\t\t\tint u=x+i, v=y+j;\n\t\t\t\t\tif(0<=u && u<DIM && 0<=v && v<DIM) flag[u][v]=true;\nint main() {\n\tbool flag[DIM][DIM];\n\tfor (int i=0; i<DIM; i++) for (int j=0; j<DIM; j++) flag[i][j]=false;'

In [17]:
k1 = keyboard(keyboard_low, keyboard_high, low_dict, high_dict)
k1.type_string(temp_sent)

972.7854054794448

In [18]:
k1.keyboard_low

[['`', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '=', '<back>'],
 ['\t', 'q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '[', ']', '\\'],
 ['<caps>', 'a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';', "'", '\n'],
 ['<shift>', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

Everything works

## Build cpp dataset

In [ ]:
import os
from tqdm import tqdm
count = 0
directory = "./validated"
sentences = []
for filename in tqdm(os.listdir(directory)):
    f = os.path.join(directory, filename)
    df = pd.read_csv(f, sep='`')
    temp = df['code']
    temp_sent = '\n'.join(temp)
    sentences.append(temp_sent)
    count += 1

In [ ]:
cpp_programs = pd.DataFrame(sentences)
cpp_programs.head()

,0
0,"point operator-(point p1,point p2)\n p1.x-=p2..."
1,"int main() {\n map<char, int> m;\n m['6'..."
2,"int main()\n\tint n,s[1001],cnt=0;\n\tscanf(""%..."
3,int main() {\n\tcin >> n;\n\tfor (int i = 0; i...
4,"int main()\n char a, b, c, d, kozir;\n ..."


In [ ]:
cpp_programs.columns = ['text']

In [ ]:
cpp_programs.head()

,text
0,"point operator-(point p1,point p2)\n p1.x-=p2..."
1,"int main() {\n map<char, int> m;\n m['6'..."
2,"int main()\n\tint n,s[1001],cnt=0;\n\tscanf(""%..."
3,int main() {\n\tcin >> n;\n\tfor (int i = 0; i...
4,"int main()\n char a, b, c, d, kozir;\n ..."


In [ ]:
len(sentences), len(cpp_programs)

(119989, 119989)

In [ ]:
cpp_programs.to_csv("./cpp_programs.csv")

In [ ]:
test_df = pd.read_csv("./cpp_programs.csv", index_col=0)
test_df.head()

,text
0,"point operator-(point p1,point p2)\n p1.x-=p2..."
1,"int main() {\n map<char, int> m;\n m['6'..."
2,"int main()\n\tint n,s[1001],cnt=0;\n\tscanf(""%..."
3,int main() {\n\tcin >> n;\n\tfor (int i = 0; i...
4,"int main()\n char a, b, c, d, kozir;\n ..."
